**Add the scraped university ranking data and see if it can enrich dataset**

In [ ]:
uni_df = pd.read_csv('D:\\OneDrive\\Documents\\PERSONAL\\PERSONAL DEVELOPMENT\\DATA SCIENCE\\Personal Project Portfolio\\PP00017_Prediction of the Success of Start-Ups Using Unbiased Classification ML Models\\test.csv')
uni_df

In [ ]:
## Create a column for rank and rename the rank_name column
import re
#     tweet =tweet.lower()# Lowercasing all the letters
#     tweet= re.sub("@[A-Za-z0-9_]+","", tweet) # Remove all the mentions
#     tweet = re.sub("#[A-Za-z0-9_]+","", tweet) # Remove all the hashtags
#     tweet = re.sub(r"http\S+", "", tweet) # Remove the URL
#     tweet = re.sub(r"www.\S+", "", tweet) # Remove the URL
#     tweet= re.sub('[()!?]', ' ', tweet) # Remove punctuations
#     tweet = re.sub('\[.*?\]',' ', tweet) # Remove punctuations
#     tweet= re.sub("[^a-z0-9]"," ", tweet) # Remove all non-alphanumeric characters
for row, col in enumerate(uni_df['rank_name']):
    uni_df.loc[row, 'name'] = re.sub("[^a-zA-Z' ']","", col)
    uni_df.loc[row, 'rank'] = re.sub("[^0-9–' ']","", col)
    uni_df.loc[row, 'rank'] = re.sub(r"(\d+)–","", uni_df.loc[row, 'rank'])
    uni_df.loc[row, 'rank'] = re.sub("[^0-9]","", uni_df.loc[row, 'rank'])

    

In [ ]:
pd.set_option('display.max_rows',len(uni_df))
uni_df

In [ ]:
# Rearrange the column arrangement
cols = ['rank', 'name', 'country', 'rank_name']
uni_df = uni_df[cols]
uni_df.head(2)


In [ ]:
# Drop the rank_name column
uni_df.drop('rank_name', axis=1, inplace=True)

In [ ]:
# rename columns
uni_df.rename(columns={'name': 'institution_name'}, inplace=True)
uni_df.head(2)

In [ ]:
# Save the dataset
uni_df.to_csv('D:\\OneDrive\\Documents\\PERSONAL\\PERSONAL DEVELOPMENT\\DATA SCIENCE\\Personal Project Portfolio\\PP00017_Prediction of the Success of Start-Ups Using Unbiased Classification ML Models\\data\\uni_rankings.csv', index=False)

In [ ]:
fit_and_score_m3d(medium_3_data, models)

In [ ]:
# helper function for nog data
def fit_and_score_nog(dataset, models):
    
    """
    Fits and evaluates models passed to it based on the data structure

    Parameters:
    -----------
    models : Dictionary of machine learning models

    Returns:
    -----------
    A dictionary of accuracy scores of each model

    """

    # Define preprocessors

    # pipeline for text data1
    text1_features = 'short_description_o'
    text1_transformer = Pipeline(steps=[
        ('vectorizer', TfidfVectorizer(stop_words="english"))
    ])

    # pipeline for text data2
    text2_features = 'description_o'
    text2_transformer = Pipeline(steps=[
        ('vectorizer', TfidfVectorizer(stop_words="english"))
    ])

    # pipeline for text data3
    text3_features = 'description_o'
    text3_transformer = Pipeline(steps=[
        ('vectorizer', TfidfVectorizer(stop_words="english"))
    ])

    # pipeline for categorical data

    no_gender_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','country_code_p',
                           'featured_job_title','institution_name','degree_type','subject','is_completed']

    no_gender_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])


    # pipeline for numeric data
    numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                         'degree_length','employee_count_min','employee_count_max']
    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


    preprocessor = ColumnTransformer(
        transformers=[
            ("tex1", text1_transformer, text1_features),
            ("tex2", text2_transformer, text2_features),
            ("tex3", text3_transformer, text3_features),
            ("num", numeric_transformer, numeric_features),
            ("nog", no_gender_transformer, no_gender_features),
        ])

    X = dataset.drop('success',axis=1)
    y = dataset['success']

    # Split into train and test dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    # Set random seed
    np.random.seed(42)

    # Make a dictionary to keeo model scores
    model_score = {}

    # Lopp through models
    for name, model in models.items():

        # Fit the model to the data
        clf = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", model)])
        clf.fit(X_train, y_train)

        model_score[f'nog_{name}'] = f'{clf.score(X_test, y_test):.3f}%'

    model_score = {k:[v] for k,v in model_score.items()}
    model_score_df =pd.DataFrame(model_score)

    return model_score_df.T




In [ ]:
fit_and_score_nog(no_gender_data, models)

In [ ]:
# helper function for full data
def fit_and_score_num(dataset, models):
    
    """
    Fits and evaluates models passed to it based on the data structure

    Parameters:
    -----------
    models : Dictionary of machine learning models

    Returns:
    -----------
    A dictionary of accuracy scores of each model

    """


    # pipeline for numeric data
    numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                         'degree_length','employee_count_min','employee_count_max']
    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


    preprocessor = ColumnTransformer(
        transformers=[

            ("num", numeric_transformer, numeric_features)

        ])

    X = dataset.drop('success',axis=1)
    y = dataset['success']

    # Split into train and test dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    # Set random seed
    np.random.seed(42)

    # Make a dictionary to keeo model scores
    model_score = {}

    # Lopp through models
    for name, model in models.items():

        # Fit the model to the data
        clf = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", model)])
        clf.fit(X_train, y_train)

        model_score[f'num_{name}'] = f'{clf.score(X_test, y_test):.3f}%'

    model_score = {k:[v] for k,v in model_score.items()}
    model_score_df =pd.DataFrame(model_score)

    return model_score_df.T


In [ ]:
fit_and_score_num(num_data, models)

## We will carry out hyperparameter tuning and also evaluate for the folllowing as stated earlier:

* Hyperparamenter tuning
* Feature Importance
* Confusion Matrix
* Cross-validation
* Precision
* Recall
* F1 Score
* Classification Report
* ROC curve
* Area under the C curve

In [ ]:
# We will use RandomizedSearchCV due to limited time to deploy Gridsearch CV

# Create a hyperparameter grid for Logistic Regression
log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
                'clf__C': np.logspace(-4,4,20),
                'clf__solver':['lbfgs','liblinear'],
               'clf__max_iter': np.arange(100,300,50)}

# Create a hyperparameter grid for SVC
svc_grid = {'clf__C': [0.1, 1, 10, 1000], 
           'clf__degree':[0, 1, 2, 3, 4, 5, 6],
           'clf__kernel':['linear', 'rbf', 'poly']}



# Create a hyperparameter grid for Random Forest
rf_grid = {'clf__n_estimators': np.arange(10,1000,50),
          'clf__max_depth': [None, 3, 5, 10],
          'clf__min_samples_split': np.arange(2, 20, 2),
          'clf__min_samples_leaf': np.arange(1, 20, 2)}

# Create a hyperparameter grid for Naive Bayes
nb_grid ={'clf__alpha':[0.1, 1, 10, 1000],
          'clf__fit_prior': [True,False]}


### Full Data

**Logistic Regression**

In [ ]:
# Run for 'fd' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = full_data.drop('success',axis=1)
y = full_data['success']

#rand_params =[log_reg_grid,svc_grid,rf_grid,nb_grid]


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=log_reg_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for log_reg fd is:{model_score}')
print(f'Best params for Log_Reg fd is:{clf_best}')


In [ ]:
# Run for 'm1d' data:

# Define preprocessors


# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])
    
    
X = medium_1_data.drop('success',axis=1)
y = medium_1_data['success']

#rand_params =[log_reg_grid,svc_grid,rf_grid,nb_grid]


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=log_reg_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for log_reg m1d is:{model_score}')
print(f'Best params for Log_Reg is m1d:{clf_best}')


In [ ]:
# Run for 'm2d' data:

# Define preprocessors


# pipeline for categorical data


# pipeline for categorical data

categorical_2_features = ['status','category_list','category_groups_list','uuid_p','gender',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_2_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat2", categorical_2_transformer, categorical_2_features),
])


X = medium_2_data.drop('success',axis=1)
y = medium_2_data['success']

#rand_params =[log_reg_grid,svc_grid,rf_grid,nb_grid]


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=log_reg_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for log_reg m2d is:{model_score}')
print(f'Best params for Log_Reg m2d is:{clf_best}')


In [ ]:
# Run for 'm3d' data:

# Define preprocessors


# pipeline for categorical data



# pipeline for categorical data

categorical_3_features = ['status','category_list','category_groups_list']

categorical_3_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat3", categorical_3_transformer, categorical_3_features),
])



X = medium_3_data.drop('success',axis=1)
y = medium_3_data['success']

#rand_params =[log_reg_grid,svc_grid,rf_grid,nb_grid]


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=log_reg_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for log_reg m3d is:{model_score}')
print(f'Best params for Log_Reg m3d is:{clf_best}')


In [ ]:
# Run for 'nog' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

no_gender_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

no_gender_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("nog", no_gender_transformer, no_gender_features),
    ])

X = no_gender_data.drop('success',axis=1)
y = no_gender_data['success']

#rand_params =[log_reg_grid,svc_grid,rf_grid,nb_grid]


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=log_reg_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for log_reg nog is:{model_score}')
print(f'Best params for Log_Reg nog is:{clf_best}')


In [ ]:
# Run for 'num' data:

# Define preprocessors


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[

        ("num", numeric_transformer, numeric_features)

    ])


X = num_data.drop('success',axis=1)
y = num_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=log_reg_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for log_reg num is:{model_score}')
print(f'Best params for Log_Reg num is:{clf_best}')


In [ ]:
# Run for 'ned' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = no_enrich_df.drop('success',axis=1)
y = no_enrich_df['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=log_reg_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for log_reg ned is:{model_score}')
print(f'Best params for Log_Reg ned is:{clf_best}')


In [ ]:
# Run for 'sed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = senti_enrich_df.drop('success',axis=1)
y = senti_enrich_df['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=log_reg_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for log_reg sed is:{model_score}')
print(f'Best params for Log_Reg sed is:{clf_best}')


In [ ]:
# Run for 'fed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = follow_enrich_df.drop('success',axis=1)
y = follow_enrich_df['success']

#rand_params =[log_reg_grid,svc_grid,rf_grid,nb_grid]


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=log_reg_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for log_reg fed is:{model_score}')
print(f'Best params for Log_Reg fed is:{clf_best}')


**SVM**

In [ ]:
# Run for 'fd' data

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = full_data.drop('success',axis=1)
y = full_data['success']

#rand_params =[log_reg_grid,svc_grid,rf_grid,nb_grid]


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=svc_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for SVC fd is:{model_score}')
print(f'Best params for SVC fd is:{clf_best}')

In [ ]:
# Run for 'm1d' data

# Define preprocessors


# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
 = medium_1_data['success']

   ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])


X = medium_1_data.drop('success',axis=1)
y = medium_1_data['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=svc_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for SVC m1d is:{model_score}')
print(f'Best params for SVC m1d is:{clf_best}')

In [ ]:
# Run for 'm2d' data
# Define preprocessors


# pipeline for categorical data

categorical_2_features = ['status','category_list','category_groups_list','uuid_p','gender',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_2_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat2", categorical_2_transformer, categorical_2_features),
])




X = medium_2_data.drop('success',axis=1)
y = medium_2_data['success']

#rand_params =[log_reg_grid,svc_grid,rf_grid,nb_grid]


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=svc_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for SVC m2d is:{model_score}')
print(f'Best params for SVC m2d is:{clf_best}')

In [ ]:
# Run for 'm3d' data:

# Define preprocessors


# pipeline for categorical data



# pipeline for categorical data

categorical_3_features = ['status','category_list','category_groups_list']

categorical_3_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat3", categorical_3_transformer, categorical_3_features),
])



X = medium_3_data.drop('success',axis=1)
y = medium_3_data['success']

#rand_params =[log_reg_grid,svc_grid,rf_grid,nb_grid]


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=svc_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for SVC m3d is:{model_score}')
print(f'Best params for SVC m3d is:{clf_best}')



In [ ]:
# Run for 'nog' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

no_gender_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

no_gender_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("nog", no_gender_transformer, no_gender_features),
    ])


X = no_gender_data.drop('success',axis=1)
y = no_gender_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=svc_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for SVC nog is:{model_score}')
print(f'Best params for SVC nog is:{clf_best}')



In [ ]:
# Run for 'num' data:

# Define preprocessors


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[

        ("num", numeric_transformer, numeric_features)

    ])


X = num_data.drop('success',axis=1)
y = num_data['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=svc_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for SVC num is:{model_score}')
print(f'Best params for SVC num is:{clf_best}')



In [ ]:
# Run for 'ned' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = no_enrich_df.drop('success',axis=1)
y = no_enrich_df['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=svc_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for SVC ned is:{model_score}')
print(f'Best params for SVC ned is:{clf_best}')



In [ ]:
# Run for 'sed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = senti_enrich_df.drop('success',axis=1)
y = senti_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=svc_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for SVC sed is:{model_score}')
print(f'Best params for SVC sed is:{clf_best}')



In [ ]:
# Run for 'fed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = follow_enrich_df.drop('success',axis=1)
y = follow_enrich_df['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=svc_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for SVC fed is:{model_score}')
print(f'Best params for SVC fed is:{clf_best}')



**Random Forest**

In [ ]:
# Run for full data


# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = full_data.drop('success',axis=1)
y = full_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=rf_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for RF fd is:{model_score}')
print(f'Best params for RF fd is:{clf_best}')

In [ ]:
# Run for 'm1d' data

# Define preprocessors


# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
   ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])


X = medium_1_data.drop('success',axis=1)
y = medium_1_data['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=rf_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for RF m1d is:{model_score}')
print(f'Best params for RF m1d is:{clf_best}')

In [ ]:

# Run for 'm2d' data
# Define preprocessors


# pipeline for categorical data

categorical_2_features = ['status','category_list','category_groups_list','uuid_p','gender',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_2_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat2", categorical_2_transformer, categorical_2_features),
])


X = medium_2_data.drop('success',axis=1)
y = medium_2_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=rf_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for RF m2d is:{model_score}')
print(f'Best params for RF m2d is:{clf_best}')

In [ ]:

# Run for 'm3d' data:

# Define preprocessors


# pipeline for categorical data



# pipeline for categorical data

categorical_3_features = ['status','category_list','category_groups_list']

categorical_3_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat3", categorical_3_transformer, categorical_3_features),
])

X = medium_3_data.drop('success',axis=1)
y = medium_3_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=rf_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for RF m3d is:{model_score}')
print(f'Best params for RF m3d is:{clf_best}')

In [ ]:
# Run for 'nog' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

no_gender_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

no_gender_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("nog", no_gender_transformer, no_gender_features),
    ])


X = no_gender_data.drop('success',axis=1)
y = no_gender_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=rf_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for RF nog is:{model_score}')
print(f'Best params for RF nog is:{clf_best}')

In [ ]:
# Run for 'num' data:

# Define preprocessors


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[

        ("num", numeric_transformer, numeric_features)

    ])


X = num_data.drop('success',axis=1)
y = num_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=rf_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for RF num is:{model_score}')
print(f'Best params for RF num is:{clf_best}')

In [ ]:

# Run for 'ned' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = no_enrich_df.drop('success',axis=1)
y = no_enrich_df['success']




# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=rf_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for RF ned is:{model_score}')
print(f'Best params for RF ned is:{clf_best}')

In [ ]:
# Run for 'sed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = senti_enrich_df.drop('success',axis=1)
y = senti_enrich_df['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=rf_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for RF sed is:{model_score}')
print(f'Best params for RF sed is:{clf_best}')

In [ ]:
# Run for 'fed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = follow_enrich_df.drop('success',axis=1)
y = follow_enrich_df['success']




# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=rf_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for RF fed is:{model_score}')
print(f'Best params for RF fed is:{clf_best}')

**Naives Bayes**

In [ ]:
# Run for full data

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])



preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = full_data.drop('success',axis=1)
y = full_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=nb_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for NB fd is:{model_score}')
print(f'Best params for NB fd is:{clf_best}')

In [ ]:

# Run for 'm1d' data

# Define preprocessors


# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
   ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])


X = medium_1_data.drop('success',axis=1)
y = medium_1_data['success']




# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=nb_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for NB m1d is:{model_score}')
print(f'Best params for NB m1d is:{clf_best}')


In [ ]:
# Run for 'm2d' data
# Define preprocessors


# pipeline for categorical data

categorical_2_features = ['status','category_list','category_groups_list','uuid_p','gender',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_2_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])



preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat2", categorical_2_transformer, categorical_2_features),
])


X = medium_2_data.drop('success',axis=1)
y = medium_2_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=nb_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for NB m2d is:{model_score}')
print(f'Best params for NB m2d is:{clf_best}')

In [ ]:
# Run for 'm3d' data:

# Define preprocessors


# pipeline for categorical data



# pipeline for categorical data

categorical_3_features = ['status','category_list','category_groups_list']

categorical_3_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat3", categorical_3_transformer, categorical_3_features),
])

X = medium_3_data.drop('success',axis=1)
y = medium_3_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=nb_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for NB m3d is:{model_score}')
print(f'Best params for NB m3d is:{clf_best}')

In [ ]:
# Run for 'nog' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

no_gender_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

no_gender_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("nog", no_gender_transformer, no_gender_features),
    ])


X = no_gender_data.drop('success',axis=1)
y = no_gender_data['success']






# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=nb_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for NB nog is:{model_score}')
print(f'Best params for NB nog is:{clf_best}')

In [ ]:

# Run for 'num' data:

# Define preprocessors


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[

        ("num", numeric_transformer, numeric_features)

    ])


X = num_data.drop('success',axis=1)
y = num_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=nb_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for NB num is:{model_score}')
print(f'Best params for NB num is:{clf_best}')

In [ ]:

# Run for 'ned' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])



preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = no_enrich_df.drop('success',axis=1)
y = no_enrich_df['success']




# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=nb_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for NB ned is:{model_score}')
print(f'Best params for NB ned is:{clf_best}')

In [ ]:
# Run for 'sed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])



preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = senti_enrich_df.drop('success',axis=1)
y = senti_enrich_df['success']




# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=nb_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for NB sed is:{model_score}')
print(f'Best params for NB sed is:{clf_best}')

In [ ]:

# Run for 'fed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])



preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = follow_enrich_df.drop('success',axis=1)
y = follow_enrich_df['success']




# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB())])

# log_reg_grid = {'clf__penalty': ['l1', 'l2', 'elasticnet', None],
#                 'clf__C': np.logspace(-4,4,20),
#                 'clf__solver':['lbfgs','liblinear'],
#                'clf__max_iter': np.arange(100,300,50)}

clf_model = RandomizedSearchCV(clf, param_distributions=nb_grid, cv=5, n_iter=20,verbose=True) 

clf_model.fit(X_train, y_train)
clf_best = clf_model.best_estimator_

model_score = clf_model.score(X_test, y_test)
best_params  =  clf_best


print(f'RandomizedSearchCV score for NB fed is:{model_score}')
print(f'Best params for NB fed is:{clf_best}')

### Final Experiments

Using the best parameters from the hyperparameter tuning, we will now model and evaluate each using the evaluation techniques for each model for each of the  9 datasets.

### Logistic Regression

In [ ]:
# Run for 'fd' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = full_data.drop('success',axis=1)
y = full_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression(C=0.23357214690901212, max_iter=150,
                                    penalty='l1', solver='liblinear'))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

plot_roc_curve(clf, X_test, y_test)

# Calculate the confusion matrix






print(f'Accuracy score for log_reg fd is {clf.score(X_test, y_test)}')



In [ ]:
sns.set(font_scale=1.5)

def plot_conf_mat(y_test, y_preds):
    """
    Plots a confusion matrix using seaborn's heatmap
    """
    
    fig, ax = plt.subplots(figsize=(3,3))
    ax = sns.heatmap(confusion_matrix(y_test, y_preds ),
                    annot=True,
                    cbar=False,
                    fmt='.0f')
    plt.xlabel('True label')
    plt.ylabel('Predicted label')
    
plot_conf_mat(y_test, y_preds)

In [ ]:
# Calculate the classification report 
print(classification_report(y_test, y_preds))

In [ ]:
# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

In [ ]:
# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

In [ ]:
# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

In [ ]:
# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

In [ ]:
# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic Reg_fd)', legend=False)

In [ ]:
# Run for 'fd' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = full_data.drop('success',axis=1)
y = full_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression(C=0.23357214690901212, max_iter=150,
                                    penalty='l1', solver='liblinear'))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

plot_roc_curve(clf, X_test, y_test)



print(f'Accuracy score for log_reg fd is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic Reg_fd)', legend=False)

In [ ]:
# Run for 'm1d' data:

# Define preprocessors


# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])
    
    
X = medium_1_data.drop('success',axis=1)
y = medium_1_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression(C=0.23357214690901212, max_iter=150,
                                    penalty='l1', solver='liblinear'))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

plot_roc_curve(clf, X_test, y_test)



print(f'Accuracy score for log_reg m1d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic Reg_m1d)', legend=False)

In [ ]:

# Run for 'm2d' data:

# Define preprocessors


# pipeline for categorical data


# pipeline for categorical data

categorical_2_features = ['status','category_list','category_groups_list','uuid_p','gender',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_2_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat2", categorical_2_transformer, categorical_2_features),
])


X = medium_2_data.drop('success',axis=1)
y = medium_2_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression(C=0.23357214690901212, max_iter=150,
                                    penalty='l1', solver='liblinear'))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

plot_roc_curve(clf, X_test, y_test)



print(f'Accuracy score for log_reg m2d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic Reg_m2d)', legend=False)

In [ ]:

# Run for 'm3d' data:

# Define preprocessors


# pipeline for categorical data



# pipeline for categorical data

categorical_3_features = ['status','category_list','category_groups_list']

categorical_3_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat3", categorical_3_transformer, categorical_3_features),
])



X = medium_3_data.drop('success',axis=1)
y = medium_3_data['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression(C=0.23357214690901212, max_iter=150,
                                    penalty='l1', solver='liblinear'))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

plot_roc_curve(clf, X_test, y_test)



print(f'Accuracy score for log_reg m3d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic Reg_m3d)', legend=False)

In [ ]:
# Run for 'nog' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

no_gender_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

no_gender_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("nog", no_gender_transformer, no_gender_features),
    ])

X = no_gender_data.drop('success',axis=1)
y = no_gender_data['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression(C=0.23357214690901212, max_iter=150,
                                    penalty='l1', solver='liblinear'))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

plot_roc_curve(clf, X_test, y_test)



print(f'Accuracy score for log_reg nog is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic Reg_nog)', legend=False)

In [ ]:
# Run for 'num' data:

# Define preprocessors


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[

        ("num", numeric_transformer, numeric_features)

    ])


X = num_data.drop('success',axis=1)
y = num_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression(C=0.23357214690901212, max_iter=150,
                                    penalty='l1', solver='liblinear'))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

plot_roc_curve(clf, X_test, y_test)



print(f'Accuracy score for log_reg num is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic Reg_num)', legend=False)

In [ ]:
# Run for 'ned' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = no_enrich_df.drop('success',axis=1)
y = no_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression(C=0.23357214690901212, max_iter=150,
                                    penalty='l1', solver='liblinear'))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

plot_roc_curve(clf, X_test, y_test)



print(f'Accuracy score for log_reg ned is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic Reg_ned)', legend=False)

In [ ]:
# Run for 'sed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = senti_enrich_df.drop('success',axis=1)
y = senti_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression(C=0.23357214690901212, max_iter=150,
                                    penalty='l1', solver='liblinear'))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

plot_roc_curve(clf, X_test, y_test)



print(f'Accuracy score for log_reg sed is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic Reg_sed)', legend=False)

In [ ]:
# Run for 'fed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = follow_enrich_df.drop('success',axis=1)
y = follow_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression(C=0.23357214690901212, max_iter=150,
                                    penalty='l1', solver='liblinear'))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

plot_roc_curve(clf, X_test, y_test)



print(f'Accuracy score for log_reg fed is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic Reg_fed)', legend=False)

## SVM

In [ ]:
# Run for 'fd' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = full_data.drop('success',axis=1)
y = full_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC(C=0.1))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for SVC fd is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (SVC_fd)', legend=False)

In [ ]:
# Run for 'm1d' data:

# Define preprocessors


# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])
    
    
X = medium_1_data.drop('success',axis=1)
y = medium_1_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC(C=0.1))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for SVC m1d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (SVC_m1d)', legend=False)

In [ ]:

# Run for 'm2d' data:

# Define preprocessors


# pipeline for categorical data


# pipeline for categorical data

categorical_2_features = ['status','category_list','category_groups_list','uuid_p','gender',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_2_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat2", categorical_2_transformer, categorical_2_features),
])


X = medium_2_data.drop('success',axis=1)
y = medium_2_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC(C=0.1))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for SVC m2d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (SVC_m2d)', legend=False)

In [ ]:

# Run for 'm3d' data:

# Define preprocessors


# pipeline for categorical data



# pipeline for categorical data

categorical_3_features = ['status','category_list','category_groups_list']

categorical_3_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat3", categorical_3_transformer, categorical_3_features),
])



X = medium_3_data.drop('success',axis=1)
y = medium_3_data['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC(C=0.1))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for SVC m3d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (SVC_m3d)', legend=False)

In [ ]:
# Run for 'nog' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

no_gender_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

no_gender_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("nog", no_gender_transformer, no_gender_features),
    ])

X = no_gender_data.drop('success',axis=1)
y = no_gender_data['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC(C=0.1))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for SVC nog is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (SVC_nog)', legend=False)

In [ ]:
# Run for 'num' data:

# Define preprocessors


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())])


preprocessor = ColumnTransformer(
    transformers=[

        ("num", numeric_transformer, numeric_features)

    ])


X = num_data.drop('success',axis=1)
y = num_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC(C=1, degree=0))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for SVC num is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (SVC_num)', legend=False)

In [ ]:
# Run for 'ned' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = no_enrich_df.drop('success',axis=1)
y = no_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC(C=0.1))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for SVC ned is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (SVC_ned)', legend=False)

In [ ]:
# Run for 'sed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = senti_enrich_df.drop('success',axis=1)
y = senti_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC(C=0.1))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for SVC sed is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (SVC_sed)', legend=False)

In [ ]:
# Run for 'fed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = follow_enrich_df.drop('success',axis=1)
y = follow_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", SVC(C=0.1))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for SVC fed is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic Reg_fed)', legend=False)

## Random Forest

In [ ]:
# Run for 'fd' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = full_data.drop('success',axis=1)
y = full_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier(max_depth=10, min_samples_leaf=3,
                                        min_samples_split=16,
                                        n_estimators=710))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for rf_fd is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (rf_fd)', legend=False)

In [ ]:
# Run for 'm1d' data:

# Define preprocessors


# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))])


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])
    
    
X = medium_1_data.drop('success',axis=1)
y = medium_1_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf",  RandomForestClassifier(max_depth=5, min_samples_split=18,
                                        n_estimators=610))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for rf_m1d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (rf_m1d)', legend=False)

In [ ]:

# Run for 'm2d' data:

# Define preprocessors


# pipeline for categorical data


# pipeline for categorical data

categorical_2_features = ['status','category_list','category_groups_list','uuid_p','gender',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_2_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat2", categorical_2_transformer, categorical_2_features),
])


X = medium_2_data.drop('success',axis=1)
y = medium_2_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier(max_depth=5, min_samples_split=18,
                                        n_estimators=610))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for rf_SVC m2d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (rf_m2d)', legend=False)

In [ ]:

# Run for 'm3d' data:

# Define preprocessors


# pipeline for categorical data



# pipeline for categorical data

categorical_3_features = ['status','category_list','category_groups_list']

categorical_3_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat3", categorical_3_transformer, categorical_3_features),
])



X = medium_3_data.drop('success',axis=1)
y = medium_3_data['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier(max_depth=5, min_samples_split=18,
                                        n_estimators=610))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for rf_m3d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (rf_m3d)', legend=False)

In [ ]:
# Run for 'nog' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

no_gender_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

no_gender_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("nog", no_gender_transformer, no_gender_features),
    ])

X = no_gender_data.drop('success',axis=1)
y = no_gender_data['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier(max_depth=10, min_samples_leaf=3,
                                        min_samples_split=16,
                                        n_estimators=710))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for rf_nog is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (rf_nog)', legend=False)

In [ ]:
# Run for 'num' data:

# Define preprocessors


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))])


preprocessor = ColumnTransformer(
    transformers=[

        ("num", numeric_transformer, numeric_features)

    ])


X = num_data.drop('success',axis=1)
y = num_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier(max_depth=3, min_samples_leaf=7,
                                        min_samples_split=6,
                                        n_estimators=560))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for rf_num is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (rf_num)', legend=False)

In [ ]:
# Run for 'ned' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = no_enrich_df.drop('success',axis=1)
y = no_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier(max_depth=10, min_samples_leaf=3,
                                        min_samples_split=16,
                                        n_estimators=710))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for rf_ned is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (rf_ned)', legend=False)

In [ ]:
# Run for 'sed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = senti_enrich_df.drop('success',axis=1)
y = senti_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf",RandomForestClassifier(max_depth=10, min_samples_split=18,
                                        n_estimators=910))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for rf_sed is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (rf_sed)', legend=False)

In [ ]:
# Run for 'fed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = follow_enrich_df.drop('success',axis=1)
y = follow_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", RandomForestClassifier(max_depth=10, min_samples_leaf=3,
                                        min_samples_split=16,
                                        n_estimators=710))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for rf_fed is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (logistic rf_fed)', legend=False)

## Naives Bayes

In [ ]:
# Run for 'fd' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = full_data.drop('success',axis=1)
y = full_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB(alpha=10))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for NB_fd is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (NB_fd)', legend=False)

In [ ]:
# Run for 'm1d' data:

# Define preprocessors


# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])
    
    
X = medium_1_data.drop('success',axis=1)
y = medium_1_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB(alpha=1000, fit_prior=False))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for NB_m1d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (NB_m1d)', legend=False)

In [ ]:

# Run for 'm2d' data:

# Define preprocessors


# pipeline for categorical data


# pipeline for categorical data

categorical_2_features = ['status','category_list','category_groups_list','uuid_p','gender',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_2_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat2", categorical_2_transformer, categorical_2_features),
])


X = medium_2_data.drop('success',axis=1)
y = medium_2_data['success']



# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB(alpha=1000))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for NB_m2d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (NB_m2d)', legend=False)

In [ ]:

# Run for 'm3d' data:

# Define preprocessors


# pipeline for categorical data



# pipeline for categorical data

categorical_3_features = ['status','category_list','category_groups_list']

categorical_3_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat3", categorical_3_transformer, categorical_3_features),
])



X = medium_3_data.drop('success',axis=1)
y = medium_3_data['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB(alpha=10, fit_prior=False))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for NB_m3d is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (NB_m3d)', legend=False)

In [ ]:
# Run for 'nog' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

no_gender_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

no_gender_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("nog", no_gender_transformer, no_gender_features),
    ])

X = no_gender_data.drop('success',axis=1)
y = no_gender_data['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB(alpha=1000, fit_prior=False))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for NB_nog is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (NB_nog)', legend=False)

In [ ]:
# Run for 'num' data:

# Define preprocessors


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[

        ("num", numeric_transformer, numeric_features)

    ])


X = num_data.drop('success',axis=1)
y = num_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB(alpha=10, fit_prior=False))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for NB_num is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (NB_num)', legend=False)

In [ ]:
# Run for 'ned' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = no_enrich_df.drop('success',axis=1)
y = no_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB(alpha=10))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for NB_ned is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (NB_ned)', legend=False)

In [ ]:
# Run for 'sed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = senti_enrich_df.drop('success',axis=1)
y = senti_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB(alpha=10))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for NB_sed is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (NB_sed)', legend=False)

In [ ]:
# Run for 'fed' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = follow_enrich_df.drop('success',axis=1)
y = follow_enrich_df['success']

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", MultinomialNB(alpha=10))])

# Fit model
clf.fit(X_train, y_train)

# Predict 

y_preds = clf.predict(X_test)

# Plot the ROC curve and calculate the AUC metric

RocCurveDisplay.from_estimator(clf, X_test, y_test)



print(f'Accuracy score for NB_fed is {clf.score(X_test, y_test)}')


sns.set(font_scale=1.5)
    
plot_conf_mat(y_test, y_preds)

# Calculate the classification report 
print(classification_report(y_test, y_preds))

# Calculate the cross validated accuracy 
cv_acc = cross_val_score(clf,  X, y, cv=5, scoring='accuracy')
cv_acc = np.mean(cv_acc)
cv_acc

# Calculate the cross validated precision
cv_precision = cross_val_score(clf,  X, y, cv=5, scoring='precision')
cv_precision = np.mean(cv_precision)
cv_precision

# Calculate the cross validated recall 
cv_recall = cross_val_score(clf,  X, y, cv=5, scoring='recall')
cv_recall = np.mean(cv_recall)
cv_recall

# Calculate the cross validated F1 score
cv_f1 = cross_val_score(clf,  X, y, cv=5, scoring='f1')
cv_f1 = np.mean(cv_f1)
cv_f1

# Visualize cross-validated metrics
cv_metrics = pd.DataFrame({'Accuracy':cv_acc,
                          'Precision': cv_precision,
                          'Recall': cv_recall,
                          'F1': cv_f1}, index=[0])
cv_metrics.T.plot.bar(title='Cross-validated metrics (NB_fed)', legend=False)

**Feature Importance**

In [ ]:
# Run for 'fd' data:

# Define preprocessors

# pipeline for text data1
text1_features = 'short_description_o'
text1_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data2
text2_features = 'description_o'
text2_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for text data3
text3_features = 'description_o'
text3_transformer = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(stop_words="english"))
])

# pipeline for categorical data

categorical_features = ['country_code_o', 'status','category_list','category_groups_list','uuid_p','gender','country_code_p',
                       'featured_job_title','institution_name','degree_type','subject','is_completed']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='not known')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# pipeline for numeric data
numeric_features = ['following', 'followers','polarity','subjectivity','rank_o','rank_p','num_events_part','per_exp_at_coy_start',
                     'degree_length','employee_count_min','employee_count_max']
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())])


preprocessor = ColumnTransformer(
    transformers=[
        ("tex1", text1_transformer, text1_features),
        ("tex2", text2_transformer, text2_features),
        ("tex3", text3_transformer, text3_features),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

X = full_data.drop('success',axis=1)
y = full_data['success']


# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# Set random seed
np.random.seed(42)

# Make a dictionary to keep model scores
model_score = {}
best_params = {}

# Fit the model to the data
clf = Pipeline(steps=[("preprocessor", preprocessor), ("clf", LogisticRegression(C=0.23357214690901212, max_iter=150,
                                    penalty='l1', solver='liblinear'))])

# Fit model
clf.fit(X_train, y_train)






In [ ]:
feature_names = clf.preprocessor.named_steps["vectorizer"].get_feature_names()